# Get access to Statsbomb data
* See link for [details about Statsbomb datasets](https://github.com/statsbomb/statsbombpy) and how to install `statsbombpy` 
* Run this notebook to create the raw datatset of all the passes  between players.

In [1]:
import pandas as pd
from statsbombpy import sb
from statsbombpy import api_client, public

In [2]:
def lineup(match_id):
    dfs_lineup = []
    lineups = public.lineups(match_id=match_id)
    for l in lineups.values():
        lineup = pd.DataFrame(l["lineup"])
        lineup["country"] = lineup.country.apply(lambda c: c['name'] if str(c) != 'nan' else '')
        lineup['team_name'] = l["team_name"]
        lineup['team_id'] = l["team_id"]
        dfs_lineup.append(lineup)
    return pd.concat(dfs_lineup)

# Get competitions

In [3]:
df_competitions = sb.competitions()
pairs_competitions = df_competitions[['competition_id', 'season_id']].values

credentials were not supplied. open data access only


In [4]:
len(pairs_competitions)

36

# Get matches

In [5]:
dfs_matches = []
for competition_id, season_id in pairs_competitions:
    try:
        df_match = sb.matches(competition_id=competition_id, season_id=season_id)
        df_match['competition_id'] = competition_id
        df_match['season_id'] = season_id
        dfs_matches.append(df_match)
    except Exception as e:
        print(f"{competition_id},{season_id} - Exception: {e}")
df_matches = pd.concat(dfs_matches)
values_matches = df_matches['match_id'].values

credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied. open data access only
16,76 - Exception: 'DataFrame' object has no attribute 'competition'
credentials were not supplied. open data access only
credentials were not supplied. open data access only
credentials were not supplied.

In [6]:
len(values_matches)

844

# Get Lineups

In [7]:
dfs_lineups = []
for match_id in values_matches[:10]:
    try:
        df_lineup = lineup(match_id)
        df_lineup['match_id'] = match_id
        dfs_lineups.append(df_lineup)
    except Exception as e:
        print(f"{match_id} - Exception: {e}")
df_lineups = pd.concat(dfs_lineups)

In [8]:
display(f"df_competitions: {df_competitions.shape}")
display(df_competitions.head(2))

'df_competitions: (36, 8)'

,competition_id,season_id,country_name,competition_name,competition_gender,season_name,match_updated,match_available
0,16,4,Europe,Champions League,male,2018/2019,2020-02-27T12:19:39.458017,2020-02-27T12:19:39.458017
1,16,1,Europe,Champions League,male,2017/2018,2020-06-11T01:24:40.306618,2020-06-11T01:24:40.306618


In [9]:
display(f"df_matches: {df_matches.shape}")
display(df_matches.head(2))

'df_matches: (844, 20)'

,match_id,match_date,kick_off,competition,season,home_team,away_team,home_score,away_score,match_status,last_updated,match_week,competition_stage,data_version,shot_fidelity_version,xy_fidelity_version,competition_id,season_id,stadium,referee
0,22912,2019-06-01,21:00:00.000,Europe - Champions League,2018/2019,Tottenham Hotspur,Liverpool,0,2,available,2019-12-16T23:09:16.168756,13,Final,1.1.0,2,2,16,4,NaN,NaN
0,18245,2018-05-26,20:45:00.000,Europe - Champions League,2017/2018,Real Madrid,Liverpool,3,1,available,2020-06-11T01:24:40.306618,7,Final,1.1.0,2,2,16,1,"{'id': 4222, 'name': 'NSK Olimpijskyj', 'count...","{'id': 727, 'name': 'M. Mažić'}"


In [10]:
display(f"df_lineups: {df_lineups.shape}")
display(df_lineups.head(2))

'df_lineups: (352, 8)'

,player_id,player_name,player_nickname,jersey_number,country,team_name,team_id,match_id
0,3043,Christian Dannemann Eriksen,Christian Eriksen,23,Denmark,Tottenham Hotspur,38,22912
1,3077,Jan Vertonghen,None,5,Belgium,Tottenham Hotspur,38,22912


In [11]:
df_lineups.columns

Index(['player_id', 'player_name', 'player_nickname', 'jersey_number',
       'country', 'team_name', 'team_id', 'match_id'],
      dtype='object')

In [12]:
df_matches.columns

Index(['match_id', 'match_date', 'kick_off', 'competition', 'season',
       'home_team', 'away_team', 'home_score', 'away_score', 'match_status',
       'last_updated', 'match_week', 'competition_stage', 'data_version',
       'shot_fidelity_version', 'xy_fidelity_version', 'competition_id',
       'season_id', 'stadium', 'referee'],
      dtype='object')

In [13]:
df_competitions.columns

Index(['competition_id', 'season_id', 'country_name', 'competition_name',
       'competition_gender', 'season_name', 'match_updated',
       'match_available'],
      dtype='object')

# Merge to one dataframe

In [14]:
df_final = df_lineups.merge(df_matches, on='match_id', how='left')
df_final = df_final.merge(df_competitions, on=['competition_id', 'season_id'], how='left')

In [15]:
display(f"df_final: {df_final.shape}")
display(df_final.head(2))

'df_final: (352, 33)'

,player_id,player_name,player_nickname,jersey_number,country,team_name,team_id,match_id,match_date,kick_off,...,competition_id,season_id,stadium,referee,country_name,competition_name,competition_gender,season_name,match_updated,match_available
0,3043,Christian Dannemann Eriksen,Christian Eriksen,23,Denmark,Tottenham Hotspur,38,22912,2019-06-01,21:00:00.000,...,16,4,NaN,NaN,Europe,Champions League,male,2018/2019,2020-02-27T12:19:39.458017,2020-02-27T12:19:39.458017
1,3077,Jan Vertonghen,None,5,Belgium,Tottenham Hotspur,38,22912,2019-06-01,21:00:00.000,...,16,4,NaN,NaN,Europe,Champions League,male,2018/2019,2020-02-27T12:19:39.458017,2020-02-27T12:19:39.458017


In [16]:
df_final.columns

Index(['player_id', 'player_name', 'player_nickname', 'jersey_number',
       'country', 'team_name', 'team_id', 'match_id', 'match_date', 'kick_off',
       'competition', 'season', 'home_team', 'away_team', 'home_score',
       'away_score', 'match_status', 'last_updated', 'match_week',
       'competition_stage', 'data_version', 'shot_fidelity_version',
       'xy_fidelity_version', 'competition_id', 'season_id', 'stadium',
       'referee', 'country_name', 'competition_name', 'competition_gender',
       'season_name', 'match_updated', 'match_available'],
      dtype='object')

# Export Player / Team

In [17]:
df_final.to_csv('../data/interim/statsbomb/player_team.csv', sep='|', index=False)